### Imports

In [2]:
from ndex2.nice_cx_network import NiceCXNetwork
import ndex2.client as nc
import ndex2
import networkx as nx
import pandas as pd
import os
import sys

print("pandas version: " + pd.__version__)

pandas version: 0.25.1


In [2]:
 def nice_cx_to_node_dataframe(nice_cx_network):
    """
    Create a Pandas DataFrame in which each row is a node and columns are node attributes.
     Example:
        ``df = nice_cx_to_node_dataframe(my_nice_cx_network) # df is now a pandas dataframe``
    :return: Pandas dataframe
    :rtype: Pandas dataframe
    """
    #TODO expand documentation
    rows = []
    node_items = None
    if sys.version_info.major == 3:
        node_items = nice_cx_network.nodes.items()
    else:
        node_items = nice_cx_network.nodes.iteritems()
    # v is the node item values
    # k is the index of the node
    attribute_names = {"name", "node_id"}
    for node_id, v in node_items:
        row = {}
        row["name"] = v.get('n')
        row["node_id"] = node_id
        for attribute in nice_cx_network.get_node_attributes(node_id):
            attribute_names.add(attribute.get('n'))
            row[attribute.get('n')] = attribute.get('v')
        rows.append(row)
    df_columns = list(attribute_names)
    return_df = pd.DataFrame(rows, columns=df_columns)
    return return_df

def jaccard_similarity_lists(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

def jaccard_similarity_sets(set1, set2):
    intersection = len(list(set1.intersection(set2)))
    union = (len(set1) + len(set2)) - intersection
    return float(intersection) / union

### The NDEx Account

In [3]:
my_account="dexterpratt"
my_password="cytoscaperules"
my_server="http://public.ndexbio.org"

if my_account == 'enter your username here':
    print('*******WARNING!!!!*******')
    print('Please change the username and password before proceeding')
else:
    try:
        my_ndex=nc.Ndex2(my_server, my_account, my_password)
        my_ndex.update_status()
        print("Success.  Please continue.")
    except Exception as inst:
        print("Could not access account %s with password %s" % (my_account, my_password))
        print(inst.args)

Success.  Please continue.


### The COVID-19 - Human Interactors Gene Set and Network

In [4]:
interactor_set = {"F2RL1", "ADAM9", "TOR1A", "PMPCA", "CISD3", "ATP13A3", "MPHOSPH10", "SLC27A2", "TIMM10B", "GORASP1", "GNG5", "EXOSC5", "EMC1", "NPC2", "PPT1", "AGPS", "LMAN2", "CDK5RAP2", "PLD3", "NUP98", "PLOD2", "NPTX1", "GCC1", "RBM28", "LARP1", "REEP5", "MDN1", "TBK1", "TM2D3", "CLCC1", "FAM8A1", "PTBP2", "POR", "RAB5C", "GTF2F2", "GRIPAP1", "VPS39", "NEU1", "MOGS", "GOLGB1", "PLAT", "AKAP9", "PTGES2", "DPY19L1", "SLC30A9", "PDE4DIP", "HS6ST2", "RNF41", "FBN2", "G3BP2", "NDFIP2", "VPS11", "BRD4", "C1orf50", "NUP210", "PIGO", "IMPDH2", "GPAA1", "GDF15", "PIGS", "ERO1B", "ZNF318", "WASHC4", "AP2A2", "ARL6IP6", "NGLY1", "ACSL3", "TOMM70", "EDEM3", "LOX", "PDZD11", "RAB18", "NARS2", "NUP88", "TARS2", "FKBP15", "INTS4", "AP3B1", "ERGIC1", "NINL", "GGCX", "SLC9A3R1", "BCKDK", "NOL10", "CHPF", "PITRM1", "CIT", "PCSK6", "TYSND1", "CSNK2B", "ATE1", "MRPS5", "DNAJC19", "JAKMIP1", "GFER", "WFS1", "SLC44A2", "UGGT2", "NUTF2", "MRPS27", "ALG11", "FOXRED2", "PVR", "ERP44", "CEP350", "DNAJC11", "SIRT5", "STOM", "NLRX1", "CYB5B", "USP54", "FAR2", "CUL2", "RAB2A", "ZNF503", "PRIM1", "IL17RA", "ITGB1", "EIF4E2", "TLE3", "DDX10", "DCAF7", "CEP43", "ANO6", "RBM41", "SBNO1", "ZDHHC5", "CYB5R3", "SLU7", "PPIL3", "UPF1", "SRP19", "TIMM9", "NUP62", "ATP5MG", "CEP250", "MTCH1", "ALG8", "DDX21", "PABPC1", "FBXL12", "BRD2", "UBXN8", "SLC30A6", "ACAD9", "RAB8A", "ELOB", "PLEKHF2", "IDE", "LARP7", "NEK9", "DNMT1", "PRKAR2B", "NAT14", "EXOSC2", "GRPEL1", "HEATR3", "PCNT", "GCC2", "SCAP", "TLE1", "BZW2", "UBAP2", "SELENOS", "AKAP8", "CLIP4", "SEPSECS", "HS2ST1", "CRTC3", "RIPK1", "RBX1", "GOLGA2", "FBN1", "ATP1B1", "NDUFAF2", "ETFA", "PABPC4", "MIB1", "RETREG3", "SMOC1", "HDAC2", "AATF", "MARK2", "CENPF", "FBLN5", "NSD2", "MRPS25", "RPL36", "PMPCB", "GGH", "GPX1", "NGDN", "QSOX2", "AP2M1", "PRIM2", "TUBGCP2", "COQ8B", "REEP6", "ERMP1", "RAB1A", "MFGE8", "RDX", "TBKBP1", "TCF12", "ERC1", "RAB10", "FAM98A", "ZC3H7A", "ABCC1", "NDUFB9", "GOLGA7", "PUSL1", "RAB7A", "SPART", "ACADM", "RAE1", "TBCA", "PLEKHA5", "NDUFAF1", "NIN", "TLE5", "COL6A1", "DPH5", "PRKAR2A", "MARK3", "MYCBP2", "TOR1AIP1", "HMOX1", "DCTPP1", "BCS1L", "ADAMTS1", "ERLEC1", "TIMM29", "NUP214", "NUP58", "MRPS2", "MARK1", "SIL1", "POLA2", "CHMP2A", "ARF6", "SUN2", "POLA1", "EIF4H", "FASTKD5", "ATP6V1A", "TAPT1", "TMEM97", "FKBP7", "STOML2", "STC2", "CEP135", "CEP112", "SRP54", "SDF2", "SNIP1", "AKAP8L", "EXOSC3", "RAB14", "ZC3H18", "TMEM39B", "CNTRL", "OS9", "ATP6AP1", "LARP4B", "FAM162A", "TIMM8B", "NUP54", "AAR2", "THTPA", "SLC30A7", "PRRC2B", "FYCO1", "PKP2", "DCAKD", "YIF1A", "GIGYF2", "GHITM", "CSDE1", "TRIM59", "HYOU1", "BAG5", "HSBP1", "SIGMAR1", "TRMT1", "MOV10", "SLC25A21", "INHBE", "GLA", "COMT", "UBAP2L", "MTARC1", "PSMD8", "FKBP10", "PRKACA", "EXOSC8", "ZYG11B", "RAP1GDS1", "ELOC", "MAT2B", "POFUT1", "POGLUT2", "HECTD1", "GOLGA3", "SAAL1", "TMED5", "MIPOL1", "RHOA", "CCDC86", "CEP68", "ECSIT"}
interaction_network_uuid = "5d97a04a-6fab-11ea-bfdc-0ac135e8bacf" 
interaction_network = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid=interaction_network_uuid, 
                                                    password=my_password, username=my_account)


### Interactions between COVID-9 proteins

### The Hierarchy

In [5]:
hierarchy_uuid = "24d586f5-7833-11ea-aaef-0ac135e8bacf" # interconnect with viral proteins
#hierarchy_uuid = "3f3f10a8-75d6-11ea-aaef-0ac135e8bacf" # diffusion
# hierarchy_uuid = "b7c7d89e-75d6-11ea-aaef-0ac135e8bacf" # neighborhood
# hierarchy_uuid = "8639b05b-75d6-11ea-aaef-0ac135e8bacf" #interconnect
hierarchy_network = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid=hierarchy_uuid, 
                                                    password=my_password, username=my_account)
# append "interactors" to its name to indicate that interactor annotations have been addded
hierarchy_network.set_name(hierarchy_network.get_name() + " - interactors")
# add the interactors as a network attribute
hierarchy_network.set_network_attribute("COVID-19-interactors", list(interactor_set), type="list_of_string")

In [6]:
#node_items = hierarchy_network.nodes.items()
#node_items
node_df = nice_cx_to_node_dataframe(hierarchy_network)
node_df

,CD_Labeled,CD_AnnotatedMembers_SourceTerm,CD_MemberList_Size,CD_AnnotatedMembers_Overlap,node_id,CD_CommunityName,CD_AnnotatedMembers_Size,name,CD_AnnotatedAlgorithm,CD_AnnotatedMembers,CD_MemberList,CD_AnnotatedMembers_SourceDB,CD_AnnotatedMembers_Pvalue,CD_NonAnnotatedMembers,CD_MemberList_LogSize
0,false,NaN,4,0.0,839455,(none),0,C790791,Annotated by gProfiler [Docker: coleslawndex/c...,NaN,OPALIN SCN3B TMTC4 URB2,NaN,NaN,NaN,2.0
1,false,NaN,4,0.0,839454,(none),0,C790790,Annotated by gProfiler [Docker: coleslawndex/c...,NaN,ENPP7 ERGIC3 IL2RA TMEM192,NaN,NaN,NaN,2.0
2,false,NaN,4,0.0,839453,(none),0,C790789,Annotated by gProfiler [Docker: coleslawndex/c...,NaN,CLCC1 MBLAC2 SPCS1 TTYH1,NaN,NaN,NaN,2.0
3,false,NaN,4,0.0,839452,(none),0,C790788,Annotated by gProfiler [Docker: coleslawndex/c...,NaN,CHRNA9 CISD2 PBXIP1 TMEM106A,NaN,NaN,NaN,2.0
4,false,NaN,4,0.0,839451,(none),0,C790787,Annotated by gProfiler [Docker: coleslawndex/c...,NaN,COA1 DNAJC11 MCOLN3 SUOX,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,true,GO:0005813,566,0.059,839270,centrosome,63,C790606,Annotated by gProfiler [Docker: coleslawndex/c...,AKAP9 APC ARHGEF10 BIRC7 CCDC92 CCNO CCT4 CCT5...,AARSD1 ABCF3 ABTB2 ACADM ACADVL AGGF1 AKAP1 AK...,GO:CC,4.961671990608699E-45,AARSD1 ABCF3 ABTB2 ACADM ACADVL AGGF1 AKAP1 AK...,9.145
186,true,GO:0031090,913,0.09,839269,organelle membrane,366,C790605,Annotated by gProfiler [Docker: coleslawndex/c...,ACAD9 ACADM ACSL3 ADGRE5 AGK AGPAT4 AGPS AIFM1...,ABCC1 ABCF2 ACAD9 ACADM ACSL3 ACVR2B ADAM11 AD...,GO:CC,7.13113736712789E-211,ABCC1 ABCF2 ACVR2B ADAM11 ADAM32 ADAM33 ADAM9 ...,9.834
187,true,GO:0012505,1487,0.104,839268,endomembrane system,564,C790604,Annotated by gProfiler [Docker: coleslawndex/c...,ACSL3 ADGRE5 ADIPOQ AGPAT4 AGRN AKAP8 AKAP9 AL...,AARSD1 AASS ABCC1 ABCF2 ABCF3 ABTB2 ACAD9 ACAD...,GO:CC,1.7203101692536646E-257,AARSD1 AASS ABCC1 ABCF2 ABCF3 ABTB2 ACAD9 ACAD...,10.538
188,true,CORUM:0000000,1985,0.114,839267,CORUM root,569,C790603,Annotated by gProfiler [Docker: coleslawndex/c...,AATF ADAM9 ADIPOQ AGPS AIFM1 AKAP1 AKAP8L AKAP...,AAR2 AARSD1 AASS AATF ABCC1 ABCF2 ABCF3 ABT1 A...,CORUM,8.171002669219583E-238,AAR2 AARSD1 AASS ABCC1 ABCF2 ABCF3 ABT1 ABTB2 ...,10.955


### Scoring

In [7]:
def in_community(row, interactors):
    #print(len(row.get("CD_MemberList")))
    #result = list(set(row.get("CD_MemberList")).intersection(interactors))
    #if (len(result) > 0):
    #    print(result)
    return list(set(row.get("CD_MemberList").split()).intersection(interactors))

def community_overlap(row, interactors):
    intersection = len(set(row.get("CD_MemberList").split()).intersection(interactors))
    memberlist_size = int(row.get("CD_MemberList_Size"))
    if isinstance(memberlist_size, (int, float)):
        return intersection / memberlist_size
    else:
        return 0

def jacquard_interactors(row, interactors):
    return jaccard_similarity_sets(set(row.get("CD_MemberList").split()), interactors)
                            
node_df["CD_InteractorMembers"] = node_df.apply(lambda row: in_community(row, interactor_set), axis=1)
                            
node_df["CD_InteractorMemberOverlap"] = node_df.apply(lambda row: community_overlap(row, interactor_set), axis=1)
node_df

# append columns + name them

,CD_Labeled,CD_AnnotatedMembers_SourceTerm,CD_MemberList_Size,CD_AnnotatedMembers_Overlap,node_id,CD_CommunityName,CD_AnnotatedMembers_Size,name,CD_AnnotatedAlgorithm,CD_AnnotatedMembers,CD_MemberList,CD_AnnotatedMembers_SourceDB,CD_AnnotatedMembers_Pvalue,CD_NonAnnotatedMembers,CD_MemberList_LogSize,CD_InteractorMembers,CD_InteractorMemberOverlap
0,false,NaN,4,0.0,839455,(none),0,C790791,Annotated by gProfiler [Docker: coleslawndex/c...,NaN,OPALIN SCN3B TMTC4 URB2,NaN,NaN,NaN,2.0,[],0.000000
1,false,NaN,4,0.0,839454,(none),0,C790790,Annotated by gProfiler [Docker: coleslawndex/c...,NaN,ENPP7 ERGIC3 IL2RA TMEM192,NaN,NaN,NaN,2.0,[],0.000000
2,false,NaN,4,0.0,839453,(none),0,C790789,Annotated by gProfiler [Docker: coleslawndex/c...,NaN,CLCC1 MBLAC2 SPCS1 TTYH1,NaN,NaN,NaN,2.0,[CLCC1],0.250000
3,false,NaN,4,0.0,839452,(none),0,C790788,Annotated by gProfiler [Docker: coleslawndex/c...,NaN,CHRNA9 CISD2 PBXIP1 TMEM106A,NaN,NaN,NaN,2.0,[],0.000000
4,false,NaN,4,0.0,839451,(none),0,C790787,Annotated by gProfiler [Docker: coleslawndex/c...,NaN,COA1 DNAJC11 MCOLN3 SUOX,NaN,NaN,NaN,2.0,[DNAJC11],0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,true,GO:0005813,566,0.059,839270,centrosome,63,C790606,Annotated by gProfiler [Docker: coleslawndex/c...,AKAP9 APC ARHGEF10 BIRC7 CCDC92 CCNO CCT4 CCT5...,AARSD1 ABCF3 ABTB2 ACADM ACADVL AGGF1 AKAP1 AK...,GO:CC,4.961671990608699E-45,AARSD1 ABCF3 ABTB2 ACADM ACADVL AGGF1 AKAP1 AK...,9.145,"[PMPCA, TYSND1, POLA1, PRKACA, USP54, ELOC, PS...",0.212014
186,true,GO:0031090,913,0.09,839269,organelle membrane,366,C790605,Annotated by gProfiler [Docker: coleslawndex/c...,ACAD9 ACADM ACSL3 ADGRE5 AGK AGPAT4 AGPS AIFM1...,ABCC1 ABCF2 ACAD9 ACADM ACSL3 ACVR2B ADAM11 AD...,GO:CC,7.13113736712789E-211,ABCC1 ABCF2 ACVR2B ADAM11 ADAM32 ADAM33 ADAM9 ...,9.834,"[TAPT1, ALG11, WFS1, MFGE8, INTS4, SPART, STC2...",0.173056
187,true,GO:0012505,1487,0.104,839268,endomembrane system,564,C790604,Annotated by gProfiler [Docker: coleslawndex/c...,ACSL3 ADGRE5 ADIPOQ AGPAT4 AGRN AKAP8 AKAP9 AL...,AARSD1 AASS ABCC1 ABCF2 ABCF3 ABTB2 ACAD9 ACAD...,GO:CC,1.7203101692536646E-257,AARSD1 AASS ABCC1 ABCF2 ABCF3 ABTB2 ACAD9 ACAD...,10.538,"[PMPCA, TYSND1, POLA1, PRKACA, TAPT1, ALG11, W...",0.184264
188,true,CORUM:0000000,1985,0.114,839267,CORUM root,569,C790603,Annotated by gProfiler [Docker: coleslawndex/c...,AATF ADAM9 ADIPOQ AGPS AIFM1 AKAP1 AKAP8L AKAP...,AAR2 AARSD1 AASS AATF ABCC1 ABCF2 ABCF3 ABT1 A...,CORUM,8.171002669219583E-238,AAR2 AARSD1 AASS ABCC1 ABCF2 ABCF3 ABT1 ABTB2 ...,10.955,"[PMPCA, WFS1, SRP54, AP3B1, ERGIC1, BCS1L, GIG...",0.156675


### Add New Attributes to Hierarchy

In [8]:
# for each of the new columns, iterate setting node attributes
for index, row in node_df.iterrows(): 
    hierarchy_network.set_node_attribute(row["node_id"],
                                         "CD_InteractorMembers", 
                                         row["CD_InteractorMembers"], 
                                         type="list_of_string" )
    hierarchy_network.set_node_attribute(row["node_id"],
                                         "CD_InteractorMemberOverlap", 
                                         row["CD_InteractorMemberOverlap"], 
                                         type="double" )


### Save to NDEx

In [9]:
if my_account == 'enter your username here':
    print('*******WARNING!!!!*******')
    print('Please change the username and password before proceeding')
else:
    upload_message = hierarchy_network.upload_to(my_server, my_account, my_password)
    print(upload_message)

Generating CX
http://public.ndexbio.org/v2/network/00bc8227-7842-11ea-aaef-0ac135e8bacf


In [10]:
node_df.to_excel(hierarchy_network.get_name() + " nodes.xlsx")